In [1]:
import logging

from IPython.core import display as ICD

from scripts.main import run

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
run()

2019-11-13 16:51:53,273|run                           : 20|INFO   | Reading configuration
2019-11-13 16:51:53,284|run                           : 30|INFO   | Starting SchedVisu workflow
2019-11-13 16:51:53,285|retrieve_and_save_data_from_PA: 32|INFO   | Retrieving data from PACS
2019-11-13 16:51:53,287|retrieve_and_save_data_from_PA: 44|INFO   | Processing 20191101
2019-11-13 16:51:53,291|retrieve_and_save_single_day_d: 66|INFO   | Skipping   20191101: save file found at "data\2019\2019-11\2019-11-01.pkl", nothing to do
2019-11-13 16:51:53,292|retrieve_and_save_data_from_PA: 44|INFO   | Processing 20191102
2019-11-13 16:51:53,294|retrieve_and_save_single_day_d: 70|INFO   | Processing 20191102: no save file found at "data\2019\2019-11\2019-11-02.pkl"
2019-11-13 16:51:53,296|find_studies_for_day          :102|INFO   | Processing 20191102: retrieving all studies from PACS
2019-11-13 16:51:55,556|retrieve_and_save_single_day_d: 75|WARNING| Warning at 20191102: no studies found
2019-11-13 1